In [2]:
import numpy as np

In [104]:
#Generates all of the three electron states which cannot be connected via translation
#This is not getting all of them.  I need to think a bit more.
#Seems to be missing things like |111> and |222>
def symkey3(N):
    keys = []
    for a in range(0,int(N/2)):
        for b in range(a,N-2*a-3):
            n = 0
            psi = ''
            psi += '1'
            n += 1
            for l in range(0,a):
                psi += '0'
                n += 1
            psi += '1'
            n += 1
            for l in range(0,b):
                psi += '0'
                n += 1
            psi += '1'
            n += 1
            for l in range(3+a+b,N):
                psi +=  '0'
                n += 1
            #print(a,b,psi)
            keys.append(psi)
    return keys

In [118]:
symkey3(9)

['111000000',
 '110100000',
 '110010000',
 '110001000',
 '110000100',
 '110000010',
 '101010000',
 '101001000',
 '101000100']

Question: given an arbitrary state, can we efficiently figure out its translational symetary key state?

For example, $\hat{K}|111000> = -|011001> + |110100>$

Is there an efficient way to relate $|011001> \rightarrow |110010>$?

If so, then we could act on the translational symmetary states by acting on the symmetry key.

### Brute force method:

Translate the state until it matches a key.

This takes $\mathcal{O}(N)$ translations per state.  Let $S$ be the number of electrons then there are $2S$ states for every application os $\hat{K}$ and we need to apply $\hat{K}$ to $M$ states where $M$ is the number of symmetry keys.  That means the total number of translations we will need to do is $\mathcal{O}(2 N S M)$.  This seems small until you relize that 
$$ N\times M \geq {N \choose S} $$
So finding the key in this way is the same as generating all of the states.

### Towards an efficient method

We can realize that $\hat{K}$ is always going to change the number of zeros between ones by one.  In other words, if we describe our symmetary states by a set of numbers which tells us how many zeros are between each one (for the three electron example, the keys are described by $a$ and $b$)  then $\hat{K}$ will change each of those numbers by one.

For example,
$$ \hat{K}|a=1,b=2> = |a=0, b=2> + |a=2, b=2> + |a=1, b=1> + |a=1, b=3> + |a=0,b=3> + |a=2,b=1> $$
$$ \hat{K}|010100100> = |001100100> + |100100100> + |010101000> + |010100010> + |011000100> + |010010100>$$ 

We have to be careful to avoid forbiden states and so forth but this might actually work.

Although it is redundant, I think it will be helpful to use one more label to describe the number of zeros to the end of the state.  So the previesou example becomes:
$$ \hat{K}|a=1,b=2,c=3> = |a=0, b=2,c=4> + |a=2, b=2,c=2> + |a=1, b=1,c=4> + |a=1, b=3,c=2> + |a=0,b=3,c=3> + |a=2,b=1,c=3> $$
$$ \hat{K}|010100100> = |001100100> + |100100100> + |010101000> + |010100010> + |011000100> + |010010100>$$ 

In general, we can write each symmetary state $|z>$ with $S = N-a-b-c$ particles in terms of the translation operator $T$ and the key state  $|a,b,c>$

$$ |z> = \frac{1}{2^{\frac{M}{2}}}\sum_{m=0}^{M-1} e^{i \phi}T^{m}|a,b,c>$$

The trick is that $\hat{K}$ commutes with $T$ so
$$ \hat{K}|z> = \frac{1}{2^{\frac{M}{2}}}\sum_{m=0}^{M-1} e^{i \phi}T^{m}\hat{K}|a,b,c> \frac{1}{2^{\frac{M}{2}}}\sum_{m=0}^{M-1} e^{i \phi}T^{m}\sum_{\text{key}}|\text{key}> = \sum_{\text{key}}|z_{\text{key}}>$$
where the keys in the summation can be found efficiently by applying $\hat{K}|a,b,c>$


This only works if $M$ is the same for every key state.  If that is not the case then we have to be careful.

In general, we should have 
$$ \hat{K}|z> =  \sum_{\text{key}}2^{\frac{M_{key}-M}{2}}\left(\sum_{m=0}^{M/M_{\text{key}}}e^{mi M_{key}\phi}\right)|z_{\text{key}}>$$

!!!!!!!!!!!!!!! This result should be checked carefully !!!!!!!!!!!!!!!!!!!!!

### Method for $\hat{N}$

$$ \hat{N} = \sum_{i=0}^{N-1} \hat{n}_{i\uparrow}\hat{n}_{i\downarrow} $$

We can write each symmetary state $|z>$ with $S = N-a-b-c$ down particles in terms of the translation operator $T$ and the key state  $|a,b,c>$.

$$ |z> = \frac{1}{2^{\frac{M}{2}}}\sum_{m=0}^{M-1} e^{i \phi}T^{m}|a,b,c>$$

since $T^{m}|a,b,c>$ and $T^{m'}|a',b',c'>$ are orthogonal for $\{a,b,c\} \neq \{a',b',c'\}$ regardless of $m$ and $m'$, $\hat{N}$ is diognal in the |z> basis.  We can check

$$ <z| \hat{n}_{i} |z> = \frac{1}{2^M}\sum_{m=0}^{M-1}<a,b,c|T^{m}\hat{n}_i T^{m}|a,b,c> $$ 

Now realize that there will be exactly $S$ different values for $m$ for which $<a,b,c|T^{m}\hat{n}_i T^{m}|a,b,c> = 1$ and it will be zero everywhere else.  Therefore,

$$ <z| \hat{n}_{i} |z> = \frac{S}{2^M} $$

Then we can check the whole opperator 
$$ <z_{\uparrow}z_{\downarrow}|\hat{N}|z_{\uparrow}z_{\downarrow}> =  \sum_{i=0}^{N-1}<z_\uparrow| \hat{n}_{i\uparrow}|z_\uparrow><z_{\downarrow}|\hat{n}_{i\downarrow}|z_{\downarrow}> =  \sum_{i=0}^{N-1}\frac{S}{2^M}\frac{S}{2^M} = \frac{N S^2}{4^M}$$

### Generating all of the keys

In fact we do not need to generate them before hand.  We can just start with any key state $|a,b,...>$ and repetedly apply $\hat{K}$ to it until no new states arise.  

We already have a rule for applying $\hat{K}$ the key states but we need to make sure that we do not have two labels for the same state.  For example, the state $|112>$ is the same as the state $|121>$.  We can gaurantee uniqueness by always rotating until the lowest number is first, if this still isnt unique then the next lowest number possible should be second as well, and so on.  Take another example, $|3123143>$ and $|4331231>$ are the same state.  Under this rule, that state should be written as $|123142>$ since $1$ is the smallest number and $2 < 4$. 

If there is more than one way to write the state and still get the same label then the normalization is determined by how many ways this can be done.  For example, the state $|123123123>$ has three different rotations which give the same label.  So the normalization becomes $2^{-N/(2\times3)}$.

Just to be clear, the rule for applying $\hat{K}$ to a key state is
$$ \hat{K}|a,b,c,...x> = |a\pm 1,b \mp 1,c,...x> + |a,b\pm 1,c\mp 1,...x>  + ... + |a\mp 1,b,c,...x\pm 1>$$

So the steps are:
    
    1) apply $\hat{K}$ as above.
    
    2) rewrite each state by
        
        a) finding the lowest number
        
        b) if there are multiple copies of this number then find the lowest number one space to the right of each 
        
        c) repeat until you find a unique chain
        
        d) if there is no unique chain then any chain will do and the normalization is reduced

In [119]:
pwd

'/Users/stenger/Documents/Research/Hubbard_symmetries'